In [59]:
import pymongo
import requests
from google.colab import userdata

client = pymongo.MongoClient('mongo_secret')

In [60]:
db = client.sample_mflix
collection = db.movies

In [30]:
hf_token = userdata.get('hf_read')

In [31]:
embedding_url = "https://api-inference.huggingface.co/pipeline/feature-extraction/sentence-transformers/all-MiniLM-L6-v2"

def generate_embedding(text: str) -> list[float]:

    response = requests.post(
        embedding_url,
        headers={"Authorization": f"Bearer {hf_token}"},
        json={"inputs": text})

    if response.status_code != 200:
        raise ValueError(f"Request failed with status code {response.status_code}: {response.text}")

    return response.json()

In [ ]:
for doc in collection.find({'plot':{"$exists": True}}).limit(50):
	doc['plot_embedding_hf'] = generate_embedding(doc['plot'])
	collection.replace_one({'_id': doc['_id']}, doc)

In [ ]:
query = "imaginary characters from outer space at war"
#query = "action or sci fi"

results = collection.aggregate([
  {"$vectorSearch": {
    "queryVector": generate_embedding(query),
    "path": "plot_embedding_hf",
    "numCandidates": 100,
    "limit": 4,
    "index": "movie_index",
      }}
])

In [38]:
for document in results:
    print(f'Movie Name: {document["title"]},\nMovie Plot: {document["plot"]}\n')

Movie Name: Four Sons,
Movie Plot: A family saga in which three of a Bavarian widow's sons go to war for Germany and the fourth goes to America, Germany's eventual opponent.

Movie Name: The Strong Man,
Movie Plot: A meek Belgian soldier (Harry Langdon) fighting in World War I receives penpal letters and a photo from "Mary Brown", an American girl he has never met. He becomes infatuated with her by ...

Movie Name: The Four Horsemen of the Apocalypse,
Movie Plot: An extended family split up in France and Germany find themselves on opposing sides of the battlefield during World War I.

Movie Name: In the Land of the Head Hunters,
Movie Plot: Original advertising for the film describes it as a drama of primitive life on the shores of the North Pacific...

